# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

ModuleNotFoundError: No module named 'gmaps'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
# Store csv created in part one into a DataFrame
weather_man_df = pd.read_csv("output_data/weather_man_data.csv")
weather_man_df.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Castelo Branco,100,PT,1601601136,81,39.82,-7.49,59.00,8.99
1,1,Sipe Sipe,99,BO,1601601250,45,-17.45,-66.38,62.60,6.93
2,2,Al Bardīyah,0,LY,1601601250,77,31.76,25.09,69.30,3.09
3,3,Chui,33,UY,1601601250,76,-33.70,-53.46,55.67,11.88
4,4,Puerto Ayora,98,EC,1601601250,86,-0.74,-90.35,68.00,8.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
#Configure gmaps
gmaps.configure(api_key=g_key)

NameError: name 'gmaps' is not defined

In [12]:
locations = weather_man_df[["Lat", "Lng"]].astype(float)
humidity_rate = weather_man_df["Humidity"].astype(float)

In [13]:
#Use the Lat and Lng as locations and Humidity as the weight.
# Create a Humidity Heatmap layer
fig = gmaps.figure(center=(40.0,0.0),zoom_level=2.5)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig

NameError: name 'gmaps' is not defined

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
# Suggested:A max temperature lower than 80 degrees but higher than 70. 
maxtemperature=weather_man_df.loc[weather_man_df["Max Temp"]<80,:]
mintemperature=maxtemperature.loc[maxtemperature["Max Temp"]>70,:]

#Suggested: Wind speed less than 10 mph.
wind_speed=mintemperature.loc[mintemperature["Wind Speed"]<10,:]
# Zero cloudiness.
zero_cloudiness=wind_speed.loc[wind_speed["Cloudiness"]==0,:]
zero_cloudiness_df1=pd.DataFrame(zero_cloudiness)

# Drop any rows that don't contain all three conditions. 
#You want to be sure the weather is ideal
zero_cloudiness_df=zero_cloudiness_df1.dropna(axis=0, how='any')

zero_cloudiness_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
64,64,Abnūb,0,EG,1601601266,50,27.27,31.15,78.80,9.17
65,65,Makkah al Mukarramah,0,SA,1601601266,41,21.43,39.83,79.39,2.21
75,75,Sur,0,OM,1601601157,52,22.57,59.53,79.59,1.97
136,136,Airai,0,TL,1601601274,50,-8.93,125.41,73.65,6.64
151,151,Castro,0,BR,1601601099,54,-24.79,-50.01,71.80,2.71


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df=pd.DataFrame(zero_cloudiness_df)
hotel_df['Hotel Name'] = ""
hotel_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
64,64,Abnūb,0,EG,1601601266,50,27.27,31.15,78.80,9.17,
65,65,Makkah al Mukarramah,0,SA,1601601266,41,21.43,39.83,79.39,2.21,
75,75,Sur,0,OM,1601601157,52,22.57,59.53,79.59,1.97,
136,136,Airai,0,TL,1601601274,50,-8.93,125.41,73.65,6.64,
151,151,Castro,0,BR,1601601099,54,-24.79,-50.01,71.80,2.71,


In [16]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#target_coordinates = f"{Lat},{Lng}"
#target_search = "Hotel"
target_radius = 5000
target_type = "lodging"
params = {
          "radius": target_radius,
         #"location":target_coordinates,
          "types": target_type,
          #"keyword": target_search, 
          "key": g_key
         }

NameError: name 'g_key' is not defined

In [17]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    city= row["City"]
    country=row["Country"]
    target_coordinates = f"{lat},{lng}" 
    print(f"Retrieving Results for Index {index}: {city}.")
    params['location'] = target_coordinates
    #make request and print url
    hotel_name = requests.get(base_url, params=params).json()
#    print(json.dumps(hotel_name, indent=4, sort_keys=True))
#    print(hotel_name)
    # extract results
    results = hotel_name['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Retrieving Results for Index 64: Abnūb.


NameError: name 'params' is not defined

In [10]:
hotel_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
64,64,Abnūb,0,EG,1601601266,50,27.27,31.15,78.80,9.17,
65,65,Makkah al Mukarramah,0,SA,1601601266,41,21.43,39.83,79.39,2.21,
75,75,Sur,0,OM,1601601157,52,22.57,59.53,79.59,1.97,
136,136,Airai,0,TL,1601601274,50,-8.93,125.41,73.65,6.64,
151,151,Castro,0,BR,1601601099,54,-24.79,-50.01,71.80,2.71,


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
markers = gmaps.marker_layer(locations,info_box_content=[f"Nearest Hotel:{hotel}"for hotel in hotel_info])
fig.add_layer(markers)

# Display figure
fig

NameError: name 'gmaps' is not defined